# Excel 分割器 - Power Automate 整合版

<!-- 
開發日期: 2025年1月
功能: 只套用篩選視圖，整合 Power Automate 權限設定
特色: 符合條件存取原則，不需要 Azure AD 應用程式
-->

這個版本專門設計為使用 **Power Automate Premium** 來設定 SharePoint 資料夾權限。

## 主要特性
- ✅ **不刪除任何記錄**（所有資料都保留在檔案中）
- ✅ **只套用 AutoFilter 篩選視圖**
- ✅ **保留所有資料驗證規則**
- ✅ **保留所有工作表（tabs）**
- ✅ **原始檔案永不修改**
- 🆕 **Power Automate 整合**（產生 Excel 輸入檔案）
- 🆕 **詳細流程設定說明**（HTML 格式）
- 🆕 **符合條件存取原則**（不需要 PnP PowerShell）

## 工作原理
🔍 **篩選視圖說明**：
- 使用 Excel 的 AutoFilter 功能
- 審查者打開檔案時只看到篩選後的資料
- 但所有資料實際上都還在檔案中
- 審查者可以透過清除篩選來查看所有資料

🔄 **Power Automate 整合**：
- 產生包含所有審查者資訊的 Excel 檔案
- Power Automate 讀取檔案並批次設定權限
- 使用 SharePoint 連接器（內建功能）
- 利用現有的 SSO 登入，符合企業安全原則

## 系統需求
- Windows 系統（已 SSO 登入）
- Microsoft Excel
- Power Automate Premium 授權
- SharePoint 同步資料夾
- Python 3.9 或更高版本

## 步驟 1: 安裝必要套件

In [ ]:
# 安裝必要套件
import sys
import subprocess
import platform

required_packages = [
    'pandas',
    'openpyxl',
    'ipywidgets',
    'xlsxwriter'
]

# Windows 特定套件
if platform.system() == 'Windows':
    required_packages.append('pywin32')

print("🔍 檢查套件安裝狀態...")
for package in required_packages:
    try:
        if package == 'pywin32':
            __import__('win32com.client')
        else:
            __import__(package.replace('-', '_'))
        print(f"✓ {package} 已安裝")
    except ImportError:
        print(f"📦 正在安裝 {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} 安裝完成")

print("\n✅ 所有必要套件已就緒！")

## 步驟 2: 匯入函式庫與設定

In [ ]:
import os
import shutil
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import glob
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import time
from typing import Dict, List, Optional, Tuple
import re

# 檢查 tkinter (檔案對話框)
try:
    import tkinter as tk
    from tkinter import filedialog, messagebox
    TKINTER_AVAILABLE = True
except ImportError:
    TKINTER_AVAILABLE = False

# 檢查 win32com (Windows Excel 自動化)
WIN32COM_AVAILABLE = False
if platform.system() == 'Windows':
    try:
        import win32com.client
        WIN32COM_AVAILABLE = True
    except ImportError:
        pass

print("✓ 函式庫匯入成功")
print(f"✓ 作業系統: {platform.system()}")
print(f"✓ 檔案對話框: {'可用' if TKINTER_AVAILABLE else '不可用'}")
print(f"✓ Excel 自動化: {'可用' if WIN32COM_AVAILABLE else '不可用'}")

# 全域變數
last_folder = os.path.expanduser("~")

## 步驟 3: Excel 處理功能（修正版 - 使用隱藏列）

In [ ]:
def sanitize_folder_name(name: str) -> str:
    """清理資料夾名稱，確保相容性"""
    # 移除或替換不允許的字元
    invalid_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '#', '%']
    sanitized = name.strip()
    
    for char in invalid_chars:
        sanitized = sanitized.replace(char, '_')
    
    # 限制長度
    if len(sanitized) > 255:
        sanitized = sanitized[:255].rstrip()
    
    return sanitized

def find_column(worksheet, column_name):
    """在工作表中尋找欄位"""
    for col_idx, cell in enumerate(worksheet[1], start=1):
        if cell.value == column_name:
            return col_idx
    raise ValueError(f"找不到 '{column_name}' 欄位！")

def copy_selected_documents(source_dir, dest_dir, copy_word=True, copy_pdf=True):
    """複製選定的文件類型"""
    copied_files = []
    
    if copy_word:
        word_patterns = [
            os.path.join(source_dir, "*.docx"),
            os.path.join(source_dir, "*.doc")
        ]
        
        for pattern in word_patterns:
            for file in glob.glob(pattern):
                if os.path.isfile(file):
                    dest_path = os.path.join(dest_dir, os.path.basename(file))
                    shutil.copy2(file, dest_path)
                    copied_files.append(os.path.basename(file))
    
    if copy_pdf:
        pdf_pattern = os.path.join(source_dir, "*.pdf")
        for file in glob.glob(pdf_pattern):
            if os.path.isfile(file):
                dest_path = os.path.join(dest_dir, os.path.basename(file))
                shutil.copy2(file, dest_path)
                copied_files.append(os.path.basename(file))
    
    return copied_files

def process_reviewer_excel_hide_rows(file_path, reviewer, column_name, output_folder):
    """修正版：使用隱藏列方式處理（保留檔案結構完整性）"""
    try:
        # 清理審查者名稱
        reviewer_name = sanitize_folder_name(str(reviewer).strip())
        
        # 建立審查者資料夾
        reviewer_folder = os.path.join(output_folder, reviewer_name)
        os.makedirs(reviewer_folder, exist_ok=True)
        
        # 建立新檔名
        base_name = os.path.basename(file_path)
        name_without_ext = os.path.splitext(base_name)[0]
        ext = os.path.splitext(base_name)[1]
        new_filename = f"{name_without_ext} - {reviewer_name}{ext}"
        dst_path = os.path.join(reviewer_folder, new_filename)
        
        # 先複製整個檔案
        shutil.copy2(file_path, dst_path)
        print(f"  ✓ 已複製檔案: {new_filename}")
        
        # 使用 openpyxl 處理複製的檔案
        # 重要：保留所有元素以避免檔案損壞
        wb = load_workbook(dst_path, data_only=False, keep_vba=True, keep_links=True)
        main_ws = wb.active
        
        # 尋找欄位
        col_idx = find_column(main_ws, column_name)
        
        # 找出需要隱藏的列（而非刪除）
        rows_to_hide = []
        total_rows = main_ws.max_row
        
        for row in range(2, total_rows + 1):  # 從第2列開始（跳過標題）
            cell_value = main_ws.cell(row=row, column=col_idx).value
            # 確保比較時處理空值和字串
            if str(cell_value).strip() != str(reviewer).strip():
                rows_to_hide.append(row)
        
        print(f"  ✓ 找到 {len(rows_to_hide)} 列需要隱藏")
        
        # 隱藏非相關列（保持檔案結構完整）
        for row in rows_to_hide:
            main_ws.row_dimensions[row].hidden = True
        
        # 設定自動篩選（如果尚未設定）
        if not main_ws.auto_filter.ref:
            max_col = main_ws.max_column
            filter_range = f"A1:{get_column_letter(max_col)}{total_rows}"
            main_ws.auto_filter.ref = filter_range
        
        # 套用篩選條件
        main_ws.auto_filter.add_filter_column(col_idx - 1, [str(reviewer)])
        
        # 儲存變更
        wb.save(dst_path)
        wb.close()
        
        print(f"  ✓ 已處理完成，隱藏非相關資料")
        
        return True, reviewer_folder, new_filename
        
    except Exception as e:
        print(f"❌ 處理 {reviewer} 的檔案時發生錯誤: {str(e)}")
        return False, None, None

def process_reviewer_excel_filter_only(file_path, reviewer, column_name, output_folder):
    """最安全方法：僅複製檔案並設定篩選（不隱藏或刪除任何內容）"""
    try:
        # 清理審查者名稱
        reviewer_name = sanitize_folder_name(str(reviewer).strip())
        
        # 建立審查者資料夾
        reviewer_folder = os.path.join(output_folder, reviewer_name)
        os.makedirs(reviewer_folder, exist_ok=True)
        
        # 建立新檔名
        base_name = os.path.basename(file_path)
        name_without_ext = os.path.splitext(base_name)[0]
        ext = os.path.splitext(base_name)[1]
        new_filename = f"{name_without_ext} - {reviewer_name}{ext}"
        dst_path = os.path.join(reviewer_folder, new_filename)
        
        # 複製整個檔案
        shutil.copy2(file_path, dst_path)
        print(f"  ✓ 已複製檔案: {new_filename}")
        
        # 開啟檔案設定篩選
        wb = load_workbook(dst_path, data_only=False, keep_vba=True, keep_links=True)
        main_ws = wb.active
        
        # 尋找欄位
        col_idx = find_column(main_ws, column_name)
        
        # 設定自動篩選
        if not main_ws.auto_filter.ref:
            max_row = main_ws.max_row
            max_col = main_ws.max_column
            filter_range = f"A1:{get_column_letter(max_col)}{max_row}"
            main_ws.auto_filter.ref = filter_range
        
        # 套用篩選條件（但不隱藏任何列）
        main_ws.auto_filter.add_filter_column(col_idx - 1, [str(reviewer)])
        
        # 儲存變更
        wb.save(dst_path)
        wb.close()
        
        print(f"  ✓ 已設定篩選條件為: {reviewer}")
        
        return True, reviewer_folder, new_filename
        
    except Exception as e:
        print(f"❌ 處理 {reviewer} 的檔案時發生錯誤: {str(e)}")
        return False, None, None

def process_reviewer_excel_windows_excel(file_path, reviewer, column_name, output_folder):
    """使用 Windows Excel COM 自動化（最可靠但較慢）"""
    if not WIN32COM_AVAILABLE:
        return False, None, None
        
    try:
        reviewer_name = sanitize_folder_name(str(reviewer).strip())
        reviewer_folder = os.path.join(output_folder, reviewer_name)
        os.makedirs(reviewer_folder, exist_ok=True)
        
        base_name = os.path.basename(file_path)
        name_without_ext = os.path.splitext(base_name)[0]
        new_filename = f"{name_without_ext} - {reviewer_name}.xlsx"
        dst_path = os.path.join(reviewer_folder, new_filename)
        
        # 啟動 Excel
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = False
        excel.DisplayAlerts = False
        
        try:
            # 開啟原始檔案
            wb = excel.Workbooks.Open(os.path.abspath(file_path))
            ws = wb.ActiveSheet
            
            # 另存新檔
            wb.SaveAs(os.path.abspath(dst_path))
            
            # 尋找欄位
            col_idx = None
            for col in range(1, ws.UsedRange.Columns.Count + 1):
                if ws.Cells(1, col).Value == column_name:
                    col_idx = col
                    break
            
            if not col_idx:
                raise ValueError(f"找不到欄位 '{column_name}'")
            
            # 套用篩選（使用隱藏而非刪除）
            ws.AutoFilterMode = False
            ws.UsedRange.AutoFilter(Field=col_idx, Criteria1=str(reviewer))
            
            # 儲存並關閉
            wb.Save()
            wb.Close()
            
            print(f"  ✓ 使用 Excel COM 處理完成")
            return True, reviewer_folder, new_filename
            
        finally:
            excel.Quit()
            
    except Exception as e:
        print(f"❌ Excel COM 處理失敗: {str(e)}")
        return False, None, None

print("✓ Excel 處理功能已就緒（修正版）")

In [ ]:
# 全域變數和輔助函數
import time
import gc
import pythoncom

# 全域變數
excel_com_instance = None  # 重要：全域 Excel COM 實例

def sanitize_folder_name(name: str) -> str:
    """清理資料夾名稱，確保相容性"""
    # 移除或替換不允許的字元
    invalid_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '#', '%']
    sanitized = name.strip()
    
    for char in invalid_chars:
        sanitized = sanitized.replace(char, '_')
    
    # 限制長度
    if len(sanitized) > 255:
        sanitized = sanitized[:255].rstrip()
    
    return sanitized

def find_column(worksheet, column_name):
    """在工作表中尋找欄位"""
    for col_idx, cell in enumerate(worksheet[1], start=1):
        if cell.value == column_name:
            return col_idx
    raise ValueError(f"找不到 '{column_name}' 欄位！")

def copy_selected_documents(source_dir, dest_dir, copy_word=True, copy_pdf=True):
    """複製選定的文件類型"""
    copied_files = []
    
    if copy_word:
        word_patterns = [
            os.path.join(source_dir, "*.docx"),
            os.path.join(source_dir, "*.doc")
        ]
        
        for pattern in word_patterns:
            for file in glob.glob(pattern):
                if os.path.isfile(file):
                    dest_path = os.path.join(dest_dir, os.path.basename(file))
                    shutil.copy2(file, dest_path)
                    copied_files.append(os.path.basename(file))
    
    if copy_pdf:
        pdf_pattern = os.path.join(source_dir, "*.pdf")
        for file in glob.glob(pdf_pattern):
            if os.path.isfile(file):
                dest_path = os.path.join(dest_dir, os.path.basename(file))
                shutil.copy2(file, dest_path)
                copied_files.append(os.path.basename(file))
    
    return copied_files

def initialize_excel_com():
    """初始化全域 Excel COM 實例"""
    global excel_com_instance
    if WIN32COM_AVAILABLE and excel_com_instance is None:
        try:
            # 初始化 COM
            pythoncom.CoInitialize()
            
            # 建立 Excel 實例
            excel_com_instance = win32com.client.Dispatch("Excel.Application")
            excel_com_instance.Visible = False
            excel_com_instance.DisplayAlerts = False
            excel_com_instance.ScreenUpdating = False
            excel_com_instance.EnableEvents = False
            
            print("✓ Excel COM 實例已初始化")
            return True
        except Exception as e:
            print(f"❌ 無法初始化 Excel COM: {str(e)}")
            excel_com_instance = None
            return False
    return excel_com_instance is not None

def cleanup_excel_com():
    """清理全域 Excel COM 實例"""
    global excel_com_instance
    if excel_com_instance is not None:
        try:
            # 關閉所有工作簿
            while excel_com_instance.Workbooks.Count > 0:
                excel_com_instance.Workbooks(1).Close(False)
            
            # 恢復設定
            excel_com_instance.ScreenUpdating = True
            excel_com_instance.EnableEvents = True
            excel_com_instance.DisplayAlerts = True
            
            # 結束 Excel
            excel_com_instance.Quit()
            excel_com_instance = None
            
            # 清理 COM
            pythoncom.CoUninitialize()
            
            # 強制垃圾回收
            gc.collect()
            time.sleep(0.5)
            
            print("✓ Excel COM 實例已清理")
        except Exception as e:
            print(f"⚠️ 清理 Excel COM 時發生錯誤: {str(e)}")
            excel_com_instance = None

def restart_excel_com():
    """重新啟動 Excel COM 實例"""
    cleanup_excel_com()
    time.sleep(1)  # 給系統一點時間
    return initialize_excel_com()

def process_reviewer_excel_windows_excel(file_path, reviewer, column_name, output_folder, retry_count=0):
    """
    使用 Windows Excel COM 自動化（修正版 - 只套用篩選，不刪除資料）
    
    重要特性：
    1. 不會修改原始檔案
    2. 不會刪除任何記錄
    3. 只套用篩選視圖來顯示相關資料
    4. 保留所有工作表和資料驗證
    
    處理流程：
    1. 開啟原始檔案（唯讀）
    2. 使用 SaveCopyAs 複製完整檔案到子資料夾
    3. 在複製的檔案上套用 AutoFilter
    4. AutoFilter 只會隱藏不相關的列，不會刪除任何資料
    """
    global excel_com_instance
    
    if not WIN32COM_AVAILABLE:
        return False, None, None
    
    # 確保 Excel COM 實例已初始化
    if excel_com_instance is None:
        if not initialize_excel_com():
            return False, None, None
    
    wb_source = None
    wb_dest = None
    
    try:
        reviewer_name = sanitize_folder_name(str(reviewer).strip())
        reviewer_folder = os.path.join(output_folder, reviewer_name)
        os.makedirs(reviewer_folder, exist_ok=True)
        
        base_name = os.path.basename(file_path)
        name_without_ext = os.path.splitext(base_name)[0]
        new_filename = f"{name_without_ext} - {reviewer_name}.xlsx"
        dst_path = os.path.join(reviewer_folder, new_filename)
        
        # 開啟原始檔案（唯讀，不會修改）
        wb_source = excel_com_instance.Workbooks.Open(os.path.abspath(file_path), ReadOnly=True)
        
        # 重要：使用 SaveCopyAs 保留完整的工作簿結構（包含所有資料）
        wb_source.SaveCopyAs(os.path.abspath(dst_path))
        wb_source.Close(False)
        wb_source = None
        
        print(f"  ✓ 已複製完整檔案到子資料夾（保留所有資料）")
        
        # 短暫延遲確保檔案釋放
        time.sleep(0.2)
        
        # 開啟複製的檔案進行處理
        wb_dest = excel_com_instance.Workbooks.Open(os.path.abspath(dst_path))
        
        # 找出主要資料工作表
        main_ws = None
        data_sheet_names = ['Sheet1', 'Data', '資料', 'Main', '主要']
        
        for sheet_name in data_sheet_names:
            try:
                main_ws = wb_dest.Worksheets(sheet_name)
                break
            except:
                continue
        
        if main_ws is None:
            main_ws = wb_dest.Worksheets(1)
        
        print(f"  ✓ 處理工作表: {main_ws.Name}")
        
        # 尋找欄位
        col_idx = None
        used_range = main_ws.UsedRange
        if used_range and used_range.Rows.Count > 0:
            for col in range(1, used_range.Columns.Count + 1):
                try:
                    if main_ws.Cells(1, col).Value == column_name:
                        col_idx = col
                        break
                except:
                    continue
        
        if not col_idx:
            raise ValueError(f"找不到欄位 '{column_name}'")
        
        # 重要：只套用篩選（AutoFilter），不刪除任何資料
        # AutoFilter 會在篩選視圖中隱藏不符合條件的列
        main_ws.AutoFilterMode = False
        used_range.AutoFilter(Field=col_idx, Criteria1=str(reviewer))
        
        print(f"  ✓ 已套用篩選條件（所有資料仍保留在檔案中）")
        
        # 儲存並關閉
        wb_dest.Save()
        wb_dest.Close()
        wb_dest = None
        
        print(f"  ✓ 處理完成 - 檔案包含所有資料，篩選視圖只顯示: {reviewer}")
        return True, reviewer_folder, new_filename
        
    except Exception as e:
        error_msg = str(e)
        print(f"❌ Excel COM 處理失敗: {error_msg}")
        
        # 嘗試關閉工作簿
        for wb in [wb_source, wb_dest]:
            if wb:
                try:
                    wb.Close(False)
                except:
                    pass
        
        # 如果是 COM 錯誤，嘗試重新啟動 Excel
        if "call was rejected" in error_msg.lower() or retry_count < 2:
            print(f"  ⚠️ 嘗試重新啟動 Excel COM (重試 {retry_count + 1}/2)")
            if restart_excel_com():
                return process_reviewer_excel_windows_excel(file_path, reviewer, column_name, output_folder, retry_count + 1)
        
        return False, None, None

print("✓ Excel 處理功能已就緒（Excel COM 方法）")
print("📌 重要說明：")
print("• 原始檔案不會被修改")
print("• 複製的檔案包含所有原始資料")
print("• 只套用篩選視圖，不刪除任何記錄")
print("• 審查者可以看到篩選後的資料，但所有資料都還在檔案中")

In [ ]:
# Power Automate 整合功能（使用 Premium 功能）
print("\n🔄 Power Automate 整合選項...")

def create_power_automate_input_file(processed_reviewers, output_folder, sharepoint_info):
    """
    建立 Power Automate 可讀取的 Excel 檔案
    包含所有需要設定權限的資訊
    """
    if not processed_reviewers:
        return None
    
    try:
        # 建立檔案路徑
        pa_file_path = os.path.join(output_folder, f"PowerAutomate_權限設定_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx")
        
        # 準備資料
        data = []
        for info in processed_reviewers:
            reviewer = info['reviewer']
            folder_name = os.path.basename(info['folder_path'])
            
            # 建立資料列
            data.append({
                '審查者名稱': reviewer,
                '資料夾名稱': folder_name,
                '資料夾完整路徑': info['folder_path'],
                'Email': f"{reviewer}@company.com",  # 預設格式，需要修改
                '權限等級': 'Contribute',  # 預設為參與權限
                '處理狀態': '待處理',
                '站台名稱': sharepoint_info.get('site', ''),
                '文件庫': sharepoint_info.get('library', 'Documents')
            })
        
        # 建立 DataFrame 並儲存為 Excel
        df = pd.DataFrame(data)
        df.to_excel(pa_file_path, index=False, sheet_name='權限設定清單')
        
        print(f"\n📄 已建立 Power Automate 輸入檔案: {os.path.basename(pa_file_path)}")
        return pa_file_path
        
    except Exception as e:
        print(f"❌ 建立 Power Automate 檔案時發生錯誤: {str(e)}")
        return None

def create_power_automate_instructions(output_folder):
    """
    建立 Power Automate 流程設定說明
    """
    try:
        instructions_path = os.path.join(output_folder, f"PowerAutomate_設定說明_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html")
        
        with open(instructions_path, 'w', encoding='utf-8') as f:
            f.write("""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Power Automate 權限設定流程說明</title>
    <style>
        body { font-family: 'Segoe UI', Arial, sans-serif; max-width: 1200px; margin: 0 auto; padding: 20px; background: #f5f5f5; }
        .container { background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
        h1 { color: #0078d4; border-bottom: 3px solid #0078d4; padding-bottom: 10px; }
        h2 { color: #323130; margin-top: 30px; }
        .step { background: #f3f2f1; padding: 20px; margin: 15px 0; border-radius: 5px; border-left: 4px solid #0078d4; }
        .step-number { font-size: 24px; font-weight: bold; color: #0078d4; }
        code { background: #e1e1e1; padding: 2px 6px; border-radius: 3px; font-family: 'Courier New', monospace; }
        .flow-json { background: #2d2d30; color: #d4d4d4; padding: 20px; border-radius: 5px; overflow-x: auto; }
        .warning { background: #fff3cd; border-left: 4px solid #ffc107; padding: 15px; margin: 20px 0; }
        .success { background: #d4edda; border-left: 4px solid #28a745; padding: 15px; margin: 20px 0; }
        img { max-width: 100%; height: auto; border: 1px solid #ddd; margin: 10px 0; }
        .button { background: #0078d4; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; display: inline-block; }
    </style>
</head>
<body>
    <div class="container">
        <h1>🔄 Power Automate SharePoint 權限設定流程</h1>
        <p>使用 Power Automate Premium 自動設定 SharePoint 資料夾權限</p>
        
        <div class="success">
            <strong>✅ 優勢：</strong>
            <ul>
                <li>不需要 Azure AD 應用程式註冊</li>
                <li>使用現有的 SSO 登入</li>
                <li>符合條件存取原則</li>
                <li>可重複使用和分享</li>
            </ul>
        </div>

        <h2>📋 步驟 1：準備工作</h2>
        <div class="step">
            <p><span class="step-number">1.1</span> 確認已產生 <code>PowerAutomate_權限設定_*.xlsx</code> 檔案</p>
            <p><span class="step-number">1.2</span> 開啟檔案並更新每個審查者的實際 Email 地址</p>
            <p><span class="step-number">1.3</span> 將檔案上傳到 SharePoint 文件庫（建議放在專用資料夾）</p>
        </div>

        <h2>🔧 步驟 2：建立 Power Automate 流程</h2>
        <div class="step">
            <p><span class="step-number">2.1</span> 前往 <a href="https://make.powerautomate.com" target="_blank">Power Automate</a></p>
            <p><span class="step-number">2.2</span> 點擊「建立」→「立即流程」→「手動觸發流程」</p>
            <p><span class="step-number">2.3</span> 命名為「SharePoint 資料夾權限設定」</p>
        </div>

        <h2>🏗️ 步驟 3：設定流程動作</h2>
        
        <div class="step">
            <h3>動作 1：列出 Excel 資料列</h3>
            <ul>
                <li>新增動作：<strong>Excel Online (Business)</strong> → <strong>列出資料表中存在的資料列</strong></li>
                <li>位置：選擇 SharePoint 站台</li>
                <li>文件庫：選擇存放 Excel 檔案的文件庫</li>
                <li>檔案：瀏覽並選擇上傳的 Excel 檔案</li>
                <li>資料表：選擇「權限設定清單」</li>
            </ul>
        </div>

        <div class="step">
            <h3>動作 2：套用到每個 (Apply to each)</h3>
            <ul>
                <li>新增動作：<strong>控制</strong> → <strong>套用到每個</strong></li>
                <li>選取先前步驟的輸出：選擇 <code>value</code></li>
            </ul>
        </div>

        <div class="step">
            <h3>動作 3：授與檔案或資料夾的存取權（在套用到每個內）</h3>
            <ul>
                <li>新增動作：<strong>SharePoint</strong> → <strong>授與檔案或資料夾的存取權</strong></li>
                <li>網站位址：選擇您的 SharePoint 站台</li>
                <li>項目 URL：使用動態內容，組合：
                    <code>/sites/[站台名稱]/[文件庫]/[資料夾名稱]</code>
                </li>
                <li>收件者：使用動態內容選擇 <code>Email</code></li>
                <li>角色：<code>可以編輯</code>（或根據需求選擇）</li>
                <li>傳送邀請：<code>是</code></li>
            </ul>
        </div>

        <div class="step">
            <h3>動作 4：更新 Excel 資料列（選用）</h3>
            <ul>
                <li>新增動作：<strong>Excel Online (Business)</strong> → <strong>更新資料列</strong></li>
                <li>更新「處理狀態」欄位為「已完成」</li>
                <li>記錄處理時間</li>
            </ul>
        </div>

        <h2>💾 步驟 4：儲存並測試</h2>
        <div class="step">
            <p><span class="step-number">4.1</span> 點擊「儲存」按鈕</p>
            <p><span class="step-number">4.2</span> 點擊「測試」→「手動」→「測試」</p>
            <p><span class="step-number">4.3</span> 檢查執行歷程記錄確認成功</p>
        </div>

        <div class="warning">
            <strong>⚠️ 注意事項：</strong>
            <ul>
                <li>確保資料夾已完全同步到 SharePoint</li>
                <li>Email 地址必須是組織內的有效地址</li>
                <li>大量資料夾可能需要較長處理時間</li>
                <li>建議先用少量資料測試</li>
            </ul>
        </div>

        <h2>🎯 進階選項</h2>
        <div class="step">
            <h3>錯誤處理</h3>
            <ul>
                <li>在「套用到每個」中設定「配置」→「並行控制」→ 開啟</li>
                <li>設定並行程度為 5-10（避免節流）</li>
                <li>加入「範圍」動作來捕捉錯誤</li>
            </ul>
        </div>

        <div class="step">
            <h3>通知功能</h3>
            <ul>
                <li>流程結束時發送摘要郵件</li>
                <li>包含成功/失敗數量</li>
                <li>列出任何錯誤訊息</li>
            </ul>
        </div>

        <h2>📊 監控和管理</h2>
        <div class="step">
            <p>執行後可以在 Power Automate 中查看：</p>
            <ul>
                <li>執行歷程記錄</li>
                <li>每個動作的詳細資訊</li>
                <li>任何錯誤或警告</li>
                <li>處理時間統計</li>
            </ul>
        </div>

        <div style="text-align: center; margin-top: 40px;">
            <a href="https://make.powerautomate.com" target="_blank" class="button">開啟 Power Automate</a>
        </div>
    </div>
</body>
</html>
""")
        
        print(f"\n📄 已建立 Power Automate 設定說明: {os.path.basename(instructions_path)}")
        return instructions_path
        
    except Exception as e:
        print(f"❌ 建立設定說明時發生錯誤: {str(e)}")
        return None

def open_power_automate():
    """
    開啟 Power Automate 網頁
    """
    try:
        import webbrowser
        webbrowser.open('https://make.powerautomate.com')
        print("\n🌐 已開啟 Power Automate")
        return True
    except:
        return False

print("\n💡 Power Automate 分享選項：")
print("1. 產生 Excel 輸入檔案（包含所有審查者資訊）")
print("2. 建立詳細的流程設定說明")
print("3. 使用 SharePoint 連接器自動授權")
print("4. 不需要 Azure AD 應用程式註冊")

In [ ]:
# === 檔案選擇 ===
display(HTML("<h3>📁 步驟 1: 檔案選擇</h3>"))

def select_excel_file():
    """選擇 Excel 檔案"""
    global last_folder
    if not TKINTER_AVAILABLE:
        print("❌ 檔案對話框不可用，請手動輸入檔案路徑")
        return
    
    try:
        root = tk.Tk()
        root.withdraw()
        root.lift()
        root.attributes('-topmost', True)
        
        file_path = filedialog.askopenfilename(
            title="選擇 Excel 檔案",
            filetypes=[(
                "Excel 檔案", "*.xlsx *.xls"),
                ("所有檔案", "*.*")
            ],
            initialdir=last_folder
        )
        
        root.destroy()
        
        if file_path:
            excel_file_input.value = file_path
            last_folder = os.path.dirname(file_path)
            
            # 自動設定輸出資料夾為同一目錄
            output_folder_input.value = os.path.dirname(file_path)
            
            with output:
                print(f"✓ 已選擇: {os.path.basename(file_path)}")
                print(f"✓ 輸出資料夾: {os.path.dirname(file_path)}")
            
    except Exception as e:
        print(f"❌ 選擇檔案時發生錯誤: {str(e)}")

excel_file_input = widgets.Text(
    value='',
    placeholder='選擇 Excel 檔案...',
    description='Excel 檔案:',
    layout=widgets.Layout(width='450px')
)

browse_button = widgets.Button(
    description='瀏覽...',
    layout=widgets.Layout(width='100px')
)

browse_button.on_click(lambda x: select_excel_file())

reviewer_column_input = widgets.Text(
    value='Reviewer',
    description='審查者欄位:',
    layout=widgets.Layout(width='300px')
)

output_folder_input = widgets.Text(
    value='',
    placeholder='輸出資料夾（預設為 Excel 檔案所在資料夾）',
    description='輸出資料夾:',
    layout=widgets.Layout(width='450px')
)

display(widgets.HBox([excel_file_input, browse_button]))
display(reviewer_column_input)
display(output_folder_input)

# === 選項設定 ===
display(HTML("<h3>⚙️ 步驟 2: 選項設定</h3>"))

copy_word_check = widgets.Checkbox(
    value=True,
    description='複製 Word 文件 (.doc, .docx)'
)

copy_pdf_check = widgets.Checkbox(
    value=True,
    description='複製 PDF 文件 (.pdf)'
)

display(copy_word_check)
display(copy_pdf_check)

# Windows Excel COM 檢查
if WIN32COM_AVAILABLE:
    display(HTML("""
    <div style='background-color: #e8f4f8; padding: 10px; border-radius: 5px; margin-top: 10px;'>
    <b>✅ Excel COM 方法可用</b><br>
    將使用 Windows Excel 程式處理檔案，確保最佳相容性。
    </div>
    """))
else:
    display(HTML("""
    <div style='background-color: #ffe6e6; padding: 10px; border-radius: 5px; margin-top: 10px;'>
    <b>❌ Excel COM 方法不可用</b><br>
    請確認：<br>
    1. 您在 Windows 系統上<br>
    2. 已安裝 Microsoft Excel<br>
    3. 已安裝 pywin32 套件
    </div>
    """))

# === 處理按鈕 ===
process_button = widgets.Button(
    description='處理 Excel 檔案',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px'),
    disabled=not WIN32COM_AVAILABLE  # 如果 COM 不可用則禁用
)

# === 輸出區域 ===
output = widgets.Output()

display(HTML("<br>"))
display(process_button)
display(output)

print("\n✅ 介面已就緒！（僅使用 Excel COM 方法）")

## 步驟 5: 主要處理函數

In [ ]:
def process_excel_file(button):
    """主要處理函數（含 Power Automate 整合）"""
    with output:
        clear_output()
        
        # 驗證輸入
        if not excel_file_input.value:
            print("❌ 請選擇 Excel 檔案")
            return
        
        if not WIN32COM_AVAILABLE:
            print("❌ Excel COM 方法不可用！")
            print("請確認您在 Windows 系統上並已安裝 Microsoft Excel")
            return
        
        file_path = excel_file_input.value.strip()
        column_name = reviewer_column_input.value.strip()
        output_folder = output_folder_input.value.strip() or os.path.dirname(file_path)
        
        if not os.path.exists(file_path):
            print(f"❌ 找不到檔案: {file_path}")
            return
        
        print(f"📁 處理中: {os.path.basename(file_path)}")
        print(f"📊 審查者欄位: {column_name}")
        print(f"📂 輸出資料夾: {output_folder}")
        print(f"🔧 使用方法: Excel COM")
        print("=" * 50)
        
        # 初始化 Excel COM
        if not initialize_excel_com():
            print("❌ 無法初始化 Excel COM")
            return
        
        # 開始計時
        start_time = time.time()
        
        # 收集處理成功的審查者資訊（用於分享功能）
        processed_reviewers = []
        
        try:
            # 讀取 Excel 檔案以取得審查者列表
            print("📖 讀取審查者列表...")
            df = pd.read_excel(file_path, engine='openpyxl')
            
            if column_name not in df.columns:
                print(f"❌ 找不到欄位 '{column_name}'")
                print(f"可用欄位: {', '.join(df.columns)}")
                return
            
            # 取得唯一審查者
            reviewers = df[column_name].dropna().unique().tolist()
            print(f"✓ 找到 {len(reviewers)} 位審查者")
            
            # 建立進度條
            progress = widgets.IntProgress(
                value=0,
                min=0,
                max=len(reviewers),
                description='進度:',
                bar_style='info'
            )
            display(progress)
            
            # 處理每位審查者
            processed = 0
            failed = 0
            failed_reviewers = []
            
            # 每 3 個檔案重新啟動 Excel 以避免記憶體問題
            restart_interval = 3
            
            for i, reviewer in enumerate(reviewers):
                print(f"\n📝 處理中: {reviewer} ({i+1}/{len(reviewers)})")
                
                # 定期重新啟動 Excel
                if i > 0 and i % restart_interval == 0:
                    print("  🔄 重新啟動 Excel COM...")
                    restart_excel_com()
                    time.sleep(1)
                
                # 使用 Excel COM 處理
                success, folder_path, filename = process_reviewer_excel_windows_excel(
                    file_path, reviewer, column_name, output_folder
                )
                
                if success:
                    # 記錄成功處理的審查者資訊
                    processed_reviewers.append({
                        'reviewer': reviewer,
                        'folder_path': folder_path,
                        'filename': filename
                    })
                    
                    # 複製相關文件
                    if copy_word_check.value or copy_pdf_check.value:
                        copied = copy_selected_documents(
                            os.path.dirname(file_path), folder_path,
                            copy_word=copy_word_check.value,
                            copy_pdf=copy_pdf_check.value
                        )
                        if copied:
                            print(f"  ✓ 已複製 {len(copied)} 個文件")
                    
                    processed += 1
                else:
                    failed += 1
                    failed_reviewers.append(reviewer)
                
                # 更新進度條
                progress.value = i + 1
                
                # 短暫延遲避免 COM 過載
                time.sleep(0.1)
            
            # 計算處理時間
            elapsed_time = time.time() - start_time
            
            # 總結
            print("\n" + "=" * 50)
            print(f"✅ 處理完成！")
            print(f"📊 成功處理: {processed}/{len(reviewers)} 位審查者")
            if failed > 0:
                print(f"❌ 處理失敗: {failed} 位")
                print(f"   失敗名單: {', '.join(failed_reviewers[:5])}" + 
                      (f" 等 {failed} 位" if failed > 5 else ""))
            print(f"⏱️ 處理時間: {elapsed_time:.1f} 秒")
            print(f"📁 輸出位置: {output_folder}")
            
            # Power Automate 分享功能
            if processed_reviewers:
                print("\n" + "=" * 50)
                print("🔄 準備 Power Automate 分享選項...")
                
                # 偵測 SharePoint 資訊
                sharepoint_info = get_sharepoint_url_from_path(output_folder)
                
                if sharepoint_info.get('found'):
                    print(f"✓ 偵測到 SharePoint 站台: {sharepoint_info.get('site')}")
                    print(f"✓ 文件庫: {sharepoint_info.get('library')}")
                else:
                    print("⚠️ 未偵測到 SharePoint 路徑模式")
                    sharepoint_info = {'found': False, 'site': '', 'library': 'Documents'}
                
                # 1. 建立 Power Automate Excel 輸入檔案
                pa_file = create_power_automate_input_file(processed_reviewers, output_folder, sharepoint_info)
                
                # 2. 建立 Power Automate 設定說明
                instructions = create_power_automate_instructions(output_folder)
                
                # 3. 建立手動分享指南（備用）
                guide_path = create_sharepoint_manual_guide(processed_reviewers, output_folder)
                
                # 4. 建立分享摘要
                summary_path = create_sharing_summary(processed_reviewers, output_folder, os.path.basename(file_path))
                
                # 5. 複製輸出資料夾路徑到剪貼簿
                if copy_path_to_clipboard(output_folder):
                    print(f"\n📋 已複製輸出資料夾路徑到剪貼簿")
            
            # 顯示完成訊息
            print("\n💡 後續步驟：")
            print("1. 開啟 PowerAutomate_權限設定_*.xlsx 檔案")
            print("2. 更新每個審查者的實際 Email 地址")
            print("3. 將檔案上傳到 SharePoint")
            print("4. 建立 Power Automate 流程（參考設定說明）")
            print("5. 執行流程自動設定權限")
            
            # 提供開啟資料夾的選項
            if platform.system() == 'Windows':
                print(f"\n📂 <a href='file:///{output_folder.replace(chr(92), '/')}' target='_blank'>開啟輸出資料夾</a>")
            
            # 提供開啟 Power Automate 的按鈕
            print("\n🔄 準備建立 Power Automate 流程？")
            open_pa_button = widgets.Button(
                description='開啟 Power Automate',
                button_style='success',
                layout=widgets.Layout(width='200px')
            )
            open_pa_button.on_click(lambda x: open_power_automate())
            display(open_pa_button)
            
            # 提供開啟 SharePoint 的按鈕（手動設定用）
            print("\n🌐 需要手動設定權限？")
            open_sp_button = widgets.Button(
                description='開啟 SharePoint',
                button_style='info',
                layout=widgets.Layout(width='200px')
            )
            open_sp_button.on_click(lambda x: open_sharepoint_in_edge())
            display(open_sp_button)
            
            # Power Automate 優勢提醒
            print("\n✨ Power Automate 優勢：")
            print("• 不需要 Azure AD 應用程式註冊")
            print("• 符合條件存取原則")
            print("• 使用現有的 SSO 登入")
            print("• 可重複使用和分享流程")
            
        except Exception as e:
            print(f"\n❌ 發生錯誤: {str(e)}")
            import traceback
            traceback.print_exc()
        finally:
            # 清理 Excel COM 實例
            cleanup_excel_com()

# 連接處理函數到按鈕
process_button.on_click(process_excel_file)
print("✓ 主要處理函數已就緒（含 Power Automate 整合）")

## 使用說明

### 🔍 篩選視圖工作原理

這個工具使用 Excel 的 **AutoFilter** 功能，不會刪除任何資料：

1. **所有資料都保留**
   - 複製的檔案包含完整的原始資料
   - 只是套用了篩選視圖
   - 審查者可以看到屬於他們的資料

2. **Excel COM 方法**
   - 使用 Windows Excel 程式處理
   - 套用 AutoFilter 篩選
   - 保留所有工作表和資料驗證

### 🔄 Power Automate 分享功能

處理完成後，工具會自動產生以下檔案供 Power Automate 使用：

1. **Excel 輸入檔案**
   - 包含所有審查者資訊
   - 預設 email 格式（需手動更新）
   - 資料夾名稱和路徑
   - SharePoint 站台資訊

2. **詳細設定說明（HTML）**
   - 逐步建立 Power Automate 流程
   - 包含所有必要的動作設定
   - 錯誤處理和最佳實踐

3. **手動分享指南**（備用）
   - 視覺化的操作指南
   - 一鍵複製功能
   - 適合少量資料夾手動設定

### 快速開始

1. **選擇 Excel 檔案**
   - 點擊『瀏覽...』選擇要處理的 Excel 檔案
   - 建議檔案位於 SharePoint 同步資料夾中

2. **確認設定**
   - 審查者欄位名稱（預設為 'Reviewer'）
   - 選擇要複製的文件類型（Word/PDF）

3. **處理檔案**
   - 點擊『處理 Excel 檔案』開始處理
   - 每個審查者會有獨立的資料夾和檔案

4. **設定 Power Automate**
   - 更新產生的 Excel 檔案中的 email 地址
   - 上傳檔案到 SharePoint
   - 建立 Power Automate 流程
   - 執行流程批次設定權限

### Power Automate 流程建立步驟

1. **建立新流程**
   - 開啟 Power Automate
   - 選擇「立即流程」→「手動觸發流程」

2. **加入動作**
   - Excel Online → 列出資料列
   - 控制 → 套用到每個
   - SharePoint → 授與存取權

3. **設定參數**
   - 選擇上傳的 Excel 檔案
   - 對應欄位到動作參數
   - 設定權限等級

4. **執行和監控**
   - 手動觸發流程
   - 監控執行狀態
   - 檢查錯誤訊息

### 審查者如何使用篩選後的檔案

當審查者打開檔案時：
- 🔍 會看到已套用的篩選（只顯示他們的資料）
- 📊 如需查看所有資料，可以：
  - 點擊篩選按鈕
  - 選擇「清除篩選」
  - 這樣就能看到完整資料

### 疑難排解

1. **Excel COM 錯誤**
   - 確保沒有其他 Excel 檔案開啟
   - 程式會自動重試和重啟 Excel

2. **Power Automate 設定**
   - 確保有 Premium 授權
   - 檢查 SharePoint 連接器權限
   - 使用並行控制避免節流

3. **權限設定失敗**
   - 確認 email 地址正確
   - 資料夾必須已同步到 SharePoint
   - 檢查使用者是否在組織內

### 為什麼使用 Power Automate？

**條件存取原則限制**：
- 許多企業禁止使用 PnP PowerShell 無 Azure AD 應用程式
- Power Automate 使用內建連接器，符合安全原則
- 不需要額外的應用程式註冊或權限

**優勢**：
- ✅ 使用現有 SSO 登入
- ✅ 符合企業安全原則
- ✅ 可視化流程設計
- ✅ 內建錯誤處理和重試
- ✅ 可分享和重複使用

### 權限等級建議

- **可以檢視**：只能查看檔案
- **可以編輯**：可以編輯檔案（推薦）
- **完全控制**：包含刪除權限（謹慎使用）

### 重要提醒

⚠️ **資料安全性**：
- 所有資料都還在檔案中（只是被篩選隱藏）
- SharePoint 權限控制資料夾存取
- 定期檢查和審核權限設定

📌 **使用前檢查清單**：
- [ ] Power Automate Premium 授權
- [ ] 檔案已同步到 SharePoint
- [ ] 更新所有審查者的 email
- [ ] 測試流程（先用少量資料）